<h1>Segmenting and Clustering Neighborhoods in the city of Toronto, Canada</h1>

<h3> Coursera - IBM Applied Data Science Capstone - Week 3 PART 3</h3>

<h4>Exploring and clustering the neighborhoods in Toronto.</h4>

Import libraries

In [18]:
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

# All requested packages already installed.



Get the data from Part 2

In [19]:
df_toronto2=pd.read_csv('Capstone_week3_part2.csv')
df_toronto2.head()

,Unnamed: 0,Unnamed: 0.1,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


We work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data as in a previous project. In order to segment the neighborhoods and explore them, we will essentially need a dataset that contains the boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

In [20]:
df_toronto3=df_toronto2[df_toronto2['Borough'].str.contains('Toronto')]
df_toronto4=df_toronto3.reset_index(drop=True)
df_toronto4.head()

,Unnamed: 0,Unnamed: 0.1,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,37,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,43,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Get the number of rows in the dataset:

In [21]:
df_toronto4.shape

(38, 7)

We get 38 unique data.

<h3>Clustering the data</h3>

First, we will have a look at the values according to each borough.

In [22]:
df_toronto4['Borough'].value_counts()

Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

Second, we will label each Neighbourhood according to its Borough in the dataframe.

In [23]:
df_toronto4['Cluster']=df_toronto4['Borough'].replace(to_replace=['Downtown Toronto','Central Toronto','West Toronto','East Toronto'],value=[1,2,3,4],inplace=False)
df_toronto4.head()

,Unnamed: 0,Unnamed: 0.1,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster
0,37,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
1,41,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4
2,42,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,4
3,43,43,M4M,East Toronto,Studio District,43.659526,-79.340923,4
4,44,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2


Next, we will visualize the data.

In [24]:
#Get the geographical coordinates of Toronto

address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.



Finally, let's visualize the resulting clusters.

In [25]:
# get the labeled cluster
toronto_cluster = len(df_toronto4.Cluster.unique())

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toronto_cluster)
ys = [i + x + (i*x)**2 for i in range(toronto_cluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto4['Latitude'], df_toronto4['Longitude'], df_toronto4['Neighbourhood'], df_toronto4['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

That's it!